In [ ]:
# 利用h5py读h5文件，把数据集和标签读出来
# 用pymysql与mysql连接，创建表和插入数据
import h5py
import pymysql
import numpy as np
import time

In [ ]:
def encode(s):
    return ' '.join([bin(ord(c)).replace('0b', '') for c in s])
 
def decode(s):
    return ''.join([chr(i) for i in [int(b, 2) for b in s.split(' ')]])

In [ ]:
np.set_printoptions(threshold=np.inf)# 将数组全部显示
#HDF5的读取：  
f = h5py.File(r'C:\Users\Administrator\Desktop\eyediagram\eyediagram.h5','r')   #打开h5文件  
# 可以查看所有的主键  
eye_values = []
for key in f.keys(): 
    print(key)
    print(f[key].name)      
    print(f[key].shape)      
    #print(f[key].value)
dataset = np.transpose(f['data'])# h5文件有许多字段
#比如师兄的data字段和labels字段，这样就选取特定字段
# 这里是转置，转成(201,360,460)
eye_num = dataset.shape[0]# 数据集个数
print(eye_num)
eye_size = str(dataset.shape[1:3])# 图片尺寸(360,460)
print(eye_size)
eye_value = dataset[0,:,:]# 数字那就是每层（每个图片）
# print(eye_value)
# print(len(str(eye_value)))# 就是一个图片的数组转换成字符串有多少个字符串 852120
labels = f['labels']
# print(labels[200])

# snr
snr = np.linspace(-10,10,201)
# print(snr)

In [ ]:
start = time.clock()
# 创建数据连接
db = pymysql.connect('localhost',
                     'root',
                     'zx041400414.',
                     'eyesight')
# 使用cursor()方法创建一个游标对象
cursor = db.cursor()

# 使用executr()方法执行SQL，如果表存在则删除
cursor.execute("DROP TABLE IF EXISTS eye_diagram")

# 使用预处理语句创建表
sql_table = """create table eye_diagram(
                diagram_id  int  not null auto_increment,
                eye_data MEDIUMBLOB not null,
                ber float not null,
                snr float not null,
                size VARCHAR(100) not null,
                update_time timestamp DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                PRIMARY key (diagram_id) )"""
cursor.execute(sql_table)

# sql插入语句
eye_data = []
for i in range(eye_num):
    #eye_data.append([encode(str(dataset[i,:,:])),labels[i],snr[i],eye_size])
    img_bite = encode(str(dataset[i,:,:]))
    sql_insert = "INSERT INTO eye_diagram(eye_data,ber,snr,size)VALUES('%s','%f','%f','%s')"%(img_bite,labels[i],snr[i],eye_size)
    try:
        cursor.execute(sql_insert)
        db.commit()
    except:
        db.rollback()
db.close()
end = time.clock()
print('Running time:%s Seconds'%(end-start))

In [ ]:
start = time.clock()
# 创建数据连接
db = pymysql.connect('localhost',
                     'root',
                     'zx041400414.',
                     'eyesight')
# 使用cursor()方法创建一个游标对象
cursor = db.cursor()
cursor.execute('select * from eye_diagram')
result = cursor.fetchall()
#img_str = decode(result)
#print(result)
#print(img_str)
end = time.clock()
print('Running time:%s Seconds'%(end-start))